In [ ]:
import pandas as pd
import numpy as np
import os
import sys


In [ ]:
sales_df = pd.read_csv('./dataset/vgsales.csv')
reviews_df = pd.read_csv('./dataset/games.csv')

# cleaning and preprocess
1. rename columns into proper names and remove unwanted columns -- Done
2. check, understand and handle nulls --> done  
3. check for duplicates and drop      --> done
4. assign correct datatypes           --> Done
5. create new table for teams and genres --> Done
6. validate numerical columns        --> Done
7. check for outlier's               --> Done
8. tranform columns as required      --> Done
9. Save new tables                   --> Done
9. Connect data to SQL server 

In [ ]:

gsales_names = {'Rank': 'rank', 'Name' : 'name', 'Platform' : 'platform', 'Year': 'year', 'Genre': 'genre', 'Publisher': 'publisher', 'NA_Sales' : 'na_Sales',
                'EU_Sales': 'eu_sales', 'JP_Sales': 'jp_sales', 'Other_Sales': 'other_sales', 'Global_Sales': 'global_sales'} 

reviews_names = {'Title': 'title', 'Release Date': 'release_date', 'Team': 'team', 'Rating': 'rating', 'Times Listed': 'times_listed', 'Number of Reviews': 'review_cnt', 'Genres': 
             'genres', 'Summary': 'summary', 'Reviews': 'reviews', 'Plays': 'plays', 'Backlogs': 'backlogs', 'Wishlist': 'wishlist'}
sales_df.rename(columns = gsales_names, inplace = True)
reviews_df.rename(columns = reviews_names, inplace = True)
reviews_df.drop(columns = ['Unnamed: 0', 'summary', 'reviews'], inplace = True)

In [ ]:
sales_df

In [ ]:
sales_df.drop_duplicates(inplace = True)
sales_df.groupby(['name'])['platform'].transform('count')

In [ ]:
reviews_df

In [ ]:
reviews_df['title'].value_counts().reset_index(name = 'cnt').sort_values(by = 'cnt', ascending = False).head(5)

In [ ]:
reviews_df[reviews_df['title'] == 'Doom']

In [ ]:
reviews_df[reviews_df['title'] == 'Resident Evil 2']

In [ ]:
duplicate_df = reviews_df[reviews_df.duplicated(['title', 'release_date'], keep = False)]
duplicate_df

In [ ]:
reviews_df.drop_duplicates(['title', 'release_date'], inplace = True)
reviews_df

In [ ]:
sales_df.isna().sum()

In [ ]:
sales_df.fillna({'publisher': 'Unknown'}, inplace = True)
sales_df.isna().sum()

In [ ]:
# if sales year is null then we put release year at sales year for that game
# games which are not present in reviews but are in sales are fine but games which are in reviews but not in sales are not fine

In [ ]:
sales_df[(sales_df['year'].isna()) | (sales_df['name'][-4:].str.isnumeric())]

In [ ]:
reviews_df.isna().sum()

In [ ]:
reviews_df[reviews_df['genres'].isna()]

In [ ]:
import ast
reviews_df['genres'] = reviews_df['genres'].apply(ast.literal_eval)

def normalize_team(x):
    if pd.isna(x):
        return ['Unknown']
    if isinstance(x, list):
        return x
    if isinstance(x, str):
        return ast.literal_eval(x)
    return ['Unknown']

reviews_df['team'] = reviews_df['team'].apply(normalize_team)
reviews_df['genres'] = reviews_df['genres'].apply( lambda x: 'unknown' if len(x) == 0 else x)



In [ ]:
games = reviews_df.drop(columns=['team', 'genres']).drop_duplicates()

game_teams = reviews_df[['title', 'team']].explode('team').drop_duplicates()

game_genres = reviews_df[['title', 'genres']].explode('genres').drop_duplicates()
games['release_date'] = pd.to_datetime(games['release_date'], errors = 'coerce')
games['year'] = games['release_date'].dt.year.fillna(0).astype('int32')



In [ ]:
df = sales_df.groupby(['name', 'year']).agg(
                               publisher_cnt = ('publisher', 'count'),
                               genre_cnt = ('genre', 'count')
)
df[df['genre_cnt'] > 1]

In [ ]:
game_publisher_df = sales_df[['name', 'publisher']].dropna().drop_duplicates(subset = ['name', 'publisher'])
game_platform_df = sales_df[['name', 'platform']].dropna().drop_duplicates(subset = ['name', 'platform'])
sales_df.drop_duplicates(subset=['name', 'year'], inplace = True)
sales_df

In [ ]:
gsales_genre = (
    sales_df[['name', 'genre']]
    .dropna()
    .drop_duplicates()
    .rename(columns={'name': 'title'})
)

game_genres = (
    game_genres
    .rename(columns={'genres': 'genre'})
)

games_genres = pd.concat([gsales_genre, game_genres], ignore_index=True).dropna().drop_duplicates(subset = ['title', 'genre'])
game_genres

In [ ]:
sales_df.drop(columns = ['genre', 'publisher', 'platform'], inplace = True)
sales_df

In [ ]:
games['times_listed'] = games['times_listed'].apply(
    lambda x: int(float(x[:-1]) * 1000) if isinstance(x, str) and x.lower().endswith('k')
    else int(x)
)
games['review_cnt'] = games['review_cnt'].apply(
    lambda x: int(float(x[:-1]) * 1000) if isinstance(x, str) and x.lower().endswith('k')
    else int(x)
)
games['plays'] = games['plays'].apply(
    lambda x: int(float(x[:-1]) * 1000) if isinstance(x, str) and x.lower().endswith('k')
    else int(x)
)
games['Playing'] = games['Playing'].apply(
    lambda x: int(float(x[:-1]) * 1000) if isinstance(x, str) and x.lower().endswith('k')
    else int(x)
)
games['backlogs'] = games['backlogs'].apply(
    lambda x: int(float(x[:-1]) * 1000) if isinstance(x, str) and x.lower().endswith('k')
    else int(x)
)

games['wishlist'] = games['wishlist'].apply(
    lambda x: int(float(x[:-1]) * 1000) if isinstance(x, str) and x.lower().endswith('k')
    else int(x)
)


games

In [ ]:
sales_dtype = {'rank': 'int32', 'name': 'string', 'platform': 'category', 'year': 'int32', 'genre': 'category', 'publisher': 'string'}
games_dtype = {'title': 'string', 'release_date': 'datetime', 'rating': 'float', 'times_listed': 'int32', 'review_cnt': 'int32',
               'plays': 'int32', 'Playing': 'int32', 'backlogs': 'int32', 'wishlist': 'int32'}
games = games.convert_dtypes(games_dtype)
sales_df = sales_df.convert_dtypes(sales_dtype)
sales_df.info()
games = (
    games.groupby(['title', 'year'], as_index=False)
    .agg(
        rating=('rating', 'mean'),
        times_listed=('times_listed', 'sum'),
        review_cnt=('review_cnt', 'sum'),
        plays=('plays', 'sum'),
        Playing=('Playing', 'sum'),
        backlogs=('backlogs', 'sum'),
        wishlist=('wishlist', 'sum')
    )
)

In [ ]:
games.info()

In [ ]:
numeric_cols = games.select_dtypes(exclude=['string']).columns
for col in numeric_cols:
    print(col, games[col].min(), games[col].max())

In [ ]:
numeric_cols = sales_df.select_dtypes(exclude=['string']).columns
for col in numeric_cols:
    print(col, sales_df[col].min(), sales_df[col].max())
sales_df = sales_df.rename(columns={'name': 'title'})

In [ ]:
games.to_csv('./dataset/games_ratings.csv', index = False)
game_teams.to_csv('./dataset/game_teams.csv', index = False)
game_genres.to_csv('./dataset/game_genres.csv', index = False)
sales_df.to_csv('./dataset/game_sales.csv', index = False)
game_publisher_df.to_csv('./dataset/game_publisher.csv', index = False)
game_platform_df.to_csv('./dataset/game_platform.csv', index = False)

In [ ]:
games[
    games.duplicated(subset=['title', 'year'], keep=False)
]


In [ ]:
df = pd.merge(
    games[['title', 'year']],
    sales_df,
    on=['title', 'year'],
    how='left',
    indicator=True
)

check_1_to_many = (
    df.groupby(['title', 'year'])
      .size()
      .reset_index(name='cnt')
      .query('cnt > 1')
)

check_1_to_many

In [ ]:
# Left join: keep all games, attach sales if present
merged = games.merge(sales_df, on=['title','year'], how='left')

# Check how many games didn’t find a match in sales
missing_in_sales = merged['global_sales'].isna().sum()
print(f"{missing_in_sales} games missing in sales")
# Right join: keep all sales, attach games if present
merged_rev = sales_df.merge(games, on=['title','year'], how='left')
missing_in_games = merged_rev['rating'].isna().sum()
print(f"{missing_in_games} sales rows missing games")


In [ ]:
df = pd.merge(sales_df, games, how = 'inner', on = ['title', 'year'])
df

Total games: 12,130

Total sales rows: 12,058

Common (matched) rows): 416

Games missing in sales: 699

Sales missing in games: 11,947

Conclusion:

After aggregating by (title, year), each key appears at most once in each table → 1-to-1 relationship

Missing rows are normal → relationship is optional on both sides

## SQL Server connect

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine

In [ ]:
database_name = 'videogames'
server_name = 'DESKTOP-I3SV1NI'
driver = 'ODBC Driver 17 for SQL Server'

connect_str = f'mssql+pyodbc://{server_name}/{database_name}?trusted_connection=yes&driver={driver}'
engine = create_engine(connect_str)